<a href="https://colab.research.google.com/github/longaresf/QxQ-Quantum-Computing-Capstone-Projects/blob/main/BB84_Eves_Intercept_and_Resend_Attack_AES256.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
!pip install cirq --quiet
import cirq
from random import choices
import numpy as np
import functools
import os
import base64
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import padding
print("Libraries imported successfully!")

Libraries imported successfully!


In [51]:
# Function to encrypt using AES in CBC mode
iv = os.urandom(16)
def encrypt_AES_CBC(message, secret_key, iv):
    padder = padding.PKCS7(128).padder()
    padded_data = padder.update(message.encode('utf-8'))
    padded_data += padder.finalize()

    cipher = Cipher(algorithms.AES(secret_key), modes.CBC(iv), backend=default_backend())
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(padded_data) + encryptor.finalize()

    return ciphertext

# Function to decrypt using AES in CBC mode
def decrypt_AES_CBC(ciphertext, secret_key, iv):
    decryptor = Cipher(algorithms.AES(secret_key), modes.CBC(iv), backend=default_backend()).decryptor()
    decrypted_data = decryptor.update(ciphertext) + decryptor.finalize()

    unpadder = padding.PKCS7(128).unpadder()
    unpadded_data = unpadder.update(decrypted_data)
    unpadded_data += unpadder.finalize()

    return unpadded_data.decode('utf-8')

# Print Exception message
class Eavesdropper_detected(BaseException):
    def __init__(self, message:str) -> None:
        self.message = message

    def __str__(self) -> str:
            output_message = self.message
            return output_message

In [3]:
# Ask for message input
message = str(input("Enter message to be encrypted: "))

Enter message to be encrypted: Hello world, this is an encrypted test message.


In [159]:
# The Setup
# Phase #1: Alice Sends
encode_gates = {0: cirq.I, 1: cirq.X}
basis_gates = {'Z': cirq.I, 'X': cirq.H}

num_bits = 18 * len(message)
#qubits
qubits = cirq.NamedQubit.range(num_bits, prefix = 'q')

# Step #1: Alice Randomly Chooses Bits
alice_key = choices([0, 1], k = num_bits)
  #  print('Alice\'s initial key: ', alice_key)

# Step #2: Alice Randomly Chooses Bases
alice_bases = choices(['Z', 'X'], k = num_bits)
  #  print('\nAlice\'s randomly chosen bases: ', alice_bases)

# Step #3: Alice Creates Qubits
alice_circuit = cirq.Circuit()

for bit in range(num_bits):

  encode_value = alice_key[bit]
  encode_gate = encode_gates[encode_value]

  basis_value = alice_bases[bit]
  basis_gate = basis_gates[basis_value]

  qubit = qubits[bit]
  alice_circuit.append(encode_gate(qubit))
  alice_circuit.append(basis_gate(qubit))
#print(alice_circuit)

# Step #4: Alice Sends the Qubits to Bob
# Phase #2: Eve Receives
# Step #5: Eve Randomly Chooses Bases
eve_bases = choices(['Z', 'X'], k = num_bits)
  #  print('Eve\'s randomly chosen bases: ', eve_bases)

eve_circuit = cirq.Circuit()

for bit in range(num_bits):

  basis_value = eve_bases[bit]
  basis_gate = basis_gates[basis_value]

  qubit = qubits[bit]
  eve_circuit.append(basis_gate(qubit))

# Step #6: Eve Measures the Qubits
eve_circuit.append(cirq.measure(qubits, key = 'eve key'))
eve_circuit

# Step #7: Eve Creates a Key
eve_intercept_circuit = alice_circuit + eve_circuit

sim = cirq.Simulator()
results = sim.run(eve_intercept_circuit)
eve_key = results.measurements['eve key'][0]
  #  print('\nEve\'s initial key: ', eve_key)

# Eve Repeats Step #3 to Fool Bob
alice_circuit = cirq.Circuit()

for bit in range(num_bits):

  encode_value = eve_key[bit]
  encode_gate = encode_gates[encode_value]

  qubit = qubits[bit]
  alice_circuit.append(encode_gate(qubit))
  alice_circuit.append(basis_gate(qubit))
    #print('\nAlice\'s Phase 1 circuit after Eve\'s interception:\n', alice_circuit)

# Phase #2: Bob Receives
# Step #5: Bob Randomly Chooses Bases
  bob_bases = choices(['Z', 'X'], k = num_bits)
  #  print('Bob\'s randomly chosen bases: ', bob_bases)

bob_circuit = cirq.Circuit()

for bit in range(num_bits):

  basis_value = bob_bases[bit]
  basis_gate = basis_gates[basis_value]

  qubit = qubits[bit]
  bob_circuit.append(basis_gate(qubit))

# Step #6: Bob Measures the Qubits
bob_circuit.append(cirq.measure(qubits, key = 'bob key'))
    #print(bob_circuit)
# Step #7: Bob Creates a Key
bb84_circuit = alice_circuit + bob_circuit

sim = cirq.Simulator()
results = sim.run(bb84_circuit)
bob_key = results.measurements['bob key'][0]
  #  print('\nBob\'s initial key: ', bob_key)

# Phase #3: Alice and Bob Compare
# Step #8: Alice and Bob Compare the Bases and Bits in Their Keys
AES_256_bit = 8
final_alice_key = []
final_bob_key = []
final_eve_key = []
final_bases = []
secret_key = []

num_bits_to_compare = int(AES_256_bit * .5)
if np.array_equal(alice_key[0:num_bits_to_compare], bob_key[0:num_bits_to_compare]):
  for bit in range(num_bits):
    if alice_bases[bit] == bob_bases[bit] and alice_key[bit] == bob_key[bit]:
      final_alice_key.append(alice_key[bit])
      final_bob_key.append(bob_key[bit])
      final_bases.append(alice_bases[bit])
      secret_key = final_alice_key[0:AES_256_bit]
  final_eve_key.append(eve_key[0:AES_256_bit].tolist())
else:
    raise Eavesdropper_detected("Eavesdropper was detected! Key couldn't be safely established.\n\nEve was listening, we need to use a different channel!")

print('\n\nWe can use our keys!')
print('\nAlice\'s key: ', final_alice_key)
print('Bob\'s key: ', final_bob_key)
print('Eve\'s key: ', final_eve_key)
print('final_bases: ', final_bases)
print('secret_key: ', secret_key)

# Step #9: Comprobation secret_key is 16, 24, or 32 bytes for AES-128, AES-192, or AES-256
if int(len(secret_key)) % 2 == 0:
    secret_key_final = secret_key
else:
    secret_key.append(1)
    secret_key_final = secret_key

print(f'secret_key_final: {secret_key_final}')

# Step #10: Comprobation Alice and Eve Compare Bits in Their Keys
if functools.reduce(lambda x, y : x and y, map(lambda p, q: p == q,secret_key_final,final_eve_key), True):
    print('\nThe alice_key and eve_key are the same')
else:
    print ("\nThe alice_key and eve_key are not the same")



We can use our keys!

Alice's key:  [0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0]
Bob's key:  [0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 

In [132]:
# Step #11: secret_key_final is encode in base64
string_data = str(secret_key_final)
byte_data = string_data.encode('utf-8')
print(f'byte_data: {byte_data}')
encoded_key = base64.b64encode(byte_data) # Encryption key Ensure the key is 16, 24, or 32 bytes for AES-128, AES-192, or AES-256
print(f'encoded_key: {encoded_key}')

# Encrypt the message
encrypted_text = encrypt_AES_CBC(message, encoded_key, iv)
print(f'Encrypted text: {encrypted_text}')

# Decrypt the encrypted message
decrypted_text = decrypt_AES_CBC(encrypted_text, encoded_key, iv)
print(f'Decrypted text: {decrypted_text}')

byte_data: b'[1, 1, 0, 1, 1, 0, 1, 1]'
encoded_key: b'WzEsIDEsIDAsIDEsIDEsIDAsIDEsIDFd'
Encrypted text: b'fa^A?u\x9a\xdf\xae\xfdR\xf1o\xd45\x0e\xa0\x0c\x19\xce-\x8c\xc7\xed\xb0L\xc6\xcc9#cbD\xb5\xbc2.\xa3\xe5\x13\xc9!\xe7\xa4\xa4g\xef,'
Decrypted text: Hello world, this is an encrypted test message.
